# Market Basket Analysis(Association Rules)

**Link**

https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset

**Dataset Description:**

The data file Groceries_dataset.csv has 3 columns.
It shows how each memeber bought specific item on a given day.
Here is a sample data from the table.

| Member_number | Date | itemDescription |
| :---: | :---: | :--- |
| 1808 | 21-07-2015 | tropical fruit |
| 2552 | 05-01-2015 | whole milk |
| 2300 | 19-09-2015 | pip fruit |
| 1187 | 12-12-2015 | other vegetables |

....

**Objective**

As he store owner I would like to understand the buying pattern of the customers. If there is a specific buying pattern then I'll plan to offer discounts on specifc items to boost the sale of other items which is typically bought together.


**Solution**

We can consider for this exercise that a combination of date and customer id can be considered as a single transaction.

So we'll first reorganize the data by forming transactions.
Then I'll run association rule analysis(in this case apriori) to find out right group of items which are frequently sold together.




In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings

# Temporarily ignore all warnings for this notebook session as they are creating noise
warnings.filterwarnings('ignore')

/Users/soumya/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Data Preparation

In [2]:
# Read the data from the file
df = pd.read_csv("./Groceries_dataset.csv")

# Step 1: Create the unique Transaction ID
df['Transaction_ID'] = df['Member_number'].astype(str) + '_' + df['Date'].astype(str)

# Step 2: Create a column to represent the value '1' (item was purchased). 
# This will later be used when we use pivot table function and will choose the 
# max value of an item's is_purchased(0,...,1) to denote if an item was purchased in a given transaction or not
# For now the value 1 denotes for a given transaction specific item was purchased.
df['is_purchased'] = 1

df.head()

,Member_number,Date,itemDescription,Transaction_ID,is_purchased
0,1808,21-07-2015,tropical fruit,1808_21-07-2015,1
1,2552,05-01-2015,whole milk,2552_05-01-2015,1
2,2300,19-09-2015,pip fruit,2300_19-09-2015,1
3,1187,12-12-2015,other vegetables,1187_12-12-2015,1
4,3037,01-02-2015,whole milk,3037_01-02-2015,1


In [3]:
# Safely drop the columns Member_number and Date as we have created the Transaction_ID column.
df.drop(columns=['Member_number', 'Date'], inplace=True)
df.head()

,itemDescription,Transaction_ID,is_purchased
0,tropical fruit,1808_21-07-2015,1
1,whole milk,2552_05-01-2015,1
2,pip fruit,2300_19-09-2015,1
3,other vegetables,1187_12-12-2015,1
4,whole milk,3037_01-02-2015,1


In [4]:
# Step 3: Pivot the table using `pivot_table`
# We use max as the aggregation function. If an item appears one or more times,
# the max value will be 1. If it doesn't appear, the fill_value=0 will be used.
transaction_data = df.pivot_table(
    index='Transaction_ID',       # New Row Index
    columns='itemDescription',    # New Column Headers
    values='is_purchased',        # Values to use in the cells (which are all 1s)
    aggfunc='max',                # Aggregation: max of 1 is 1, max of 0 is 0.
    fill_value=0                  # Fill NaNs (where item was not bought) with 0.
).astype(int) # Ensure all values are integers
transaction_data.head()

itemDescription,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Transaction_ID,,,,,,,,,,,,,,,,,,,,,
1000_15-03-2015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1000_24-06-2014,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1000_24-07-2015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000_25-11-2015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000_27-05-2015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14963 entries, 1000_15-03-2015 to 5000_16-11-2014
Columns: 167 entries, Instant food products to zwieback
dtypes: int64(167)
memory usage: 19.2+ MB


In [6]:
transaction_data.describe()

itemDescription,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
count,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,...,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000,14963.000000
mean,0.004010,0.021386,0.001470,0.001938,0.000200,0.000267,0.008087,0.001136,0.033950,0.021787,...,0.005347,0.003408,0.018512,0.043708,0.000535,0.023993,0.011696,0.157923,0.085879,0.004010
std,0.063199,0.144672,0.038317,0.043983,0.014159,0.016348,0.089564,0.033689,0.181108,0.145993,...,0.072927,0.058284,0.134799,0.204451,0.023117,0.153031,0.107515,0.364681,0.280194,0.063199
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Remove any transaction where nothing was purchased or any item column that is not purchased

(Note : There is no possibility of such scenario as the transaction_data was created from rows 
 where is_purchased was set to 1 )
 
So no changes are being made here based on the input data. Otherwise we could have done this data cleansing

## Find out frequent itemset

In [7]:
# First find out how many transactions are captured
print(transaction_data.shape)
print("Total transactions : ", transaction_data.shape[0])
print("Unique items : ", transaction_data.shape[1]-1)

(14963, 167)
Total transactions :  14963
Unique items :  166


In [8]:
# Since there are around 15K transactions, we'll consider any itemset, that appears less than 50 times in this 
# dataset is not significant enough to be considered for building association rule.
# So 50 can be considered roughly 0.003 times of 15K total records. So we'll use support value as 0.003

CUTOFF_SUPPORT_VALUE = 0.003
# Run the Apriori algorithm
frequent_itemsets = apriori(
    transaction_data,
    min_support=CUTOFF_SUPPORT_VALUE,
    use_colnames=True  # Use the item names instead of column indices
)

# Sort and display the frequent itemsets
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
print(frequent_itemsets.size)

print(frequent_itemsets.head(100))

432
      support                            itemsets
108  0.157923                        (whole milk)
67   0.122101                  (other vegetables)
82   0.110005                        (rolls/buns)
91   0.097106                              (soda)
109  0.085879                            (yogurt)
..        ...                                 ...
177  0.006483                (whole milk, pastry)
205  0.006349         (whole milk, shopping bags)
170  0.006282  (tropical fruit, other vegetables)
11   0.006148                          (cake bar)
62   0.006148                           (mustard)

[100 rows x 2 columns]


**Analysis**
There are total 432 frequent itemsets(some of them are single item and some are combination of multiple items)

## Generate the Association Rules

In [9]:
# Now that we have frequent items which were bough at least 50 times, 
# we'll try to find out which other items were bought atleast 15% of the times 
# if the first item/preceeding itemset was bought.
# This is done by having confidence value as 0.5


CUTOFF_CONFIDENCE = 0.15

rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=CUTOFF_CONFIDENCE
)

# rules.size

In [10]:
# Sort the rules by Lift and Confidence (to find the strongest, most non-obvious rules)
rules = rules.sort_values(['lift', 'confidence'], ascending=[False, False])

print(f"\nFound {len(rules)} association rules (min_confidence=0.15):")

# Display the rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])



Found 1 association rules (min_confidence=0.15):
      antecedents   consequents   support  confidence     lift
0  (bottled beer)  (whole milk)  0.007151    0.157817  0.99933


## Analysis
So from the analysis it seems 

**{bottled beer and whole milk}** are bought frequently. 
So to improve the sale of whole milk which might have a higher margin bottled beer can be sold at a 
discounted price.
However the lift value of 0.99 signifies they might be very close to being independent as well.